In [ ]:
# main adversarial attack toolbox 
!pip install foolbox

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 62 kB 606 kB/s 
     |████████████████████████████████| 180 kB 46.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust


# Create data

In [ ]:
# from requirements of https://github.com/hendrycks/robustness/tree/master/ImageNet-C/imagenet_c
# for natural corruptions 
%%capture
!apt install libmagickwand-dev
!pip install numpy Pillow scikit-image Wand opencv-python scipy # most likely only Wand is needed 

In [ ]:
import os
from pathlib import Path

import numpy as np
from tqdm.notebook import tqdm
import yaml 

import torch
import torchvision

from src import monochrome_corruptions 
from src.pmnist_robustness_data_utils import create_and_save_datasets


In [ ]:
data_raw_path = Path('data/raw')
data_inp_path = Path('data/input')

mnist_data_path = data_raw_path / 'mnist_pytorch'
root_data_path = data_inp_path / 'pmnist_robustness_test'

task_sequence = ['MNIST', 'pMNIST']

task_metadata = dict(
    task_sequence = task_sequence,
    num_tasks = len(task_sequence),
    root_data = str(root_data_path), 
    task_ids = ['task-%02d' %(i+1) for i in range(len(task_sequence))],
    task_paths = dict()
)

In [ ]:
%%time 
# ignore warning about `true_divide`, this was fixed by replacing these inf or nan values with 0 
root_data_path.mkdir(parents=True, exist_ok=True)

subset_mono_natcrpt = {k: v for i, (k,v) in 
                       enumerate(monochrome_corruptions.corruption_collection.items())
                       if i < 2}

                    
perturbation_actions = dict(
    type='corruptions',
    fns=subset_mono_natcrpt
)

task_ids = task_metadata['task_ids']
for i  in tqdm(range(len(task_ids))):
    task_id = task_ids[i]
    task_name = task_metadata['task_sequence'][i]
    perm_dset = 'p' in  task_name

    task_metadata['task_paths'][task_id] = create_and_save_datasets(
        mnist_data_path, 
        root_data_path, 
        task_id, 
        perm=perm_dset, 
        actions=perturbation_actions
    )


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 59.7 s, sys: 590 ms, total: 1min
Wall time: 1min 7s


In [ ]:
# save with natural corruptions first 
with open(root_data_path / 'metadata.yaml', 'w') as file:
    yaml.safe_dump(task_metadata, file, default_flow_style=False)

In [ ]:
# now write info about which attacks to use 
with open(root_data_path / 'metadata.yaml', 'r') as file:
    task_metadata = yaml.safe_load(file)


task_metadata['adversarial_attacks'] = dict(
    attacks = [
        'FGSM', 
        {'DDNAttack-1': dict(steps = 5)}, # if want to use additional args
        {'DDNAttack-2': dict(steps = 5)}, # and avoid duplicates by using '-'
    ], 
    epsilons = [0.0] \
           + (np.array([[1,2,4,8]]) * np.array([1e-4, 1e-3])[:, None]).flatten().tolist() \
           + (np.logspace(0,3.5,15,base=2.0)*1e-2).round(3).tolist() \
           + [0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
)

with open(root_data_path / 'metadata.yaml', 'w') as file:
    yaml.safe_dump(task_metadata, file, default_flow_style=False)

# Run training and testing

In [ ]:
!python src/pmnist_robustness_main.py --help

usage: pmnist_robustness_main.py [-h] [--input INPUT]
                                 [--batch-size BATCH_SIZE]
                                 [--num-workers NUM_WORKERS] [--pin-memory]
                                 [--hidden-layers HIDDEN_LAYERS [HIDDEN_LAYERS ...]]
                                 [--init INIT] [--init-width INIT_WIDTH]
                                 [--lr LR] [--meta META [META ...]]
                                 [--decay DECAY] [--gamma GAMMA]
                                 [--epochs-per-task EPOCHS_PER_TASK] [--dq]
                                 [--dq-beta DQ_BETA] [--dq-norm DQ_NORM]
                                 [--test-corruption] [--test-adversarial]
                                 [--skip-training] [--save-path SAVE_PATH]
                                 [--output-name OUTPUT_NAME] [--seed SEED]
                                 [--device DEVICE]

BNN learning several tasks in a row, metaplasticity is controlled by the
argument meta.

option

In [ ]:
torch.__version__

'1.10.0+cu111'

In [ ]:
%%time 
!python -W ignore src/pmnist_robustness_main.py \
    --input data/input/pmnist_robustness_test/metadata.yaml \
    --hidden-layers 256 256 \
    --meta 1.5 \
    --epochs-per-task 3 \
    --lr 0.005 --decay 1e-7 \
    --dq --dq-beta 6e-4 \
    --batch-size 200 \
    --test-corruption \
    --test-adversarial \
    --num-workers 1 \
    --save-path data/output/test \
    --output-name pmnist_robustness_test


>>>>>>>>>>>>>>>>>>>> EXPERIMENT CONFIG <<<<<<<<<<<<<<<<<<

areas:
- null
batch_size: 200
beaker: false
bin_path: false
decay: 1.0e-07
device: 0
dq: true
dq_beta: 0.0006
dq_norm: null
epochs_per_task: 3
ewc: false
ewc_lambda: 0.0
fb: null
gamma: 1.0
hidden_layers:
- 256
- 256
in_size: 784
init: uniform
init_width: 0.1
input: data/input/pmnist_robustness_test/metadata.yaml
interleaved: false
lr: 0.005
meta:
- 1.5
n_bk: null
net: bnn
norm: bn
num_workers: 1
out_size: 10
output_name: pmnist_robustness_test
pin_memory: false
ratios:
- null
rnd_consolidation: false
save_path: data/output/test
scenario: task
seed: null
si: false
si_lambda: 0.0
skip_training: false
test_adversarial: true
test_corruption: true

The functions for adversarial attacks are: 
{'FGSM': LinfFastGradientAttack(rel_stepsize=1.0, abs_stepsize=None, steps=1, random_start=False), 'DDNAttack-1': DDNAttack(init_epsilon=1.0, steps=5, gamma=0.05), 'DDNAttack-2': DDNAttack(init_epsilon=1.0, steps=5, gamma=0.05)}
>>>>>>>>>>>>>>>

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('data/output/test/pmnist_robustness_test/perf_forget.csv')

,net,arch,lr,meta,task_order,task_id,task_epoch,glob_epoch,train_acc,train_loss,train_time,test_acc::task-01,test_loss::task-01,test_time::task-01,test_acc::task-02,test_loss::task-02,test_time::task-02
0,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",1,task-01,1,1,95.49,0.193137,6.825022,94.73,0.207686,0.984283,11.44,3.349643,0.958000
1,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",1,task-01,2,2,96.08,0.140717,6.714049,95.50,0.165287,0.977431,10.39,3.899330,1.016732
2,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",1,task-01,3,3,97.08,0.103155,6.772294,96.06,0.134931,0.936277,11.21,4.677013,0.986772
3,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",2,task-02,1,4,96.00,0.138035,6.645840,74.60,0.808280,0.961755,95.11,0.164211,1.003633
4,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",2,task-02,2,5,96.48,0.116709,6.702873,56.08,1.474226,0.942826,95.58,0.146849,1.015049
5,bnn,256-256,0.005,"{'l1': 1.5, 'l2': 1.5, 'l3': 1.5}",2,task-02,3,6,97.63,0.080111,6.571939,52.05,1.571511,0.931926,96.27,0.121247,0.978283


In [ ]:
pd.read_csv('data/output/test/pmnist_robustness_test/perf_robust.csv')

,source_task,data_key,train_phase,test_acc,test_loss,test_time
0,task-01,corruptions::gaussian_noise,task-01,43.69,1.919849,0.963970
1,task-01,corruptions::gaussian_noise,task-02,40.53,2.283718,0.971744
2,task-01,corruptions::shot_noise,task-01,51.70,1.584707,0.934585
3,task-01,corruptions::shot_noise,task-02,44.47,1.998707,0.925248
4,task-01,original::test,task-01,52.05,1.571511,0.955073
5,task-01,original::test,task-02,44.30,1.986235,0.983666
6,task-02,corruptions::gaussian_noise,task-01,73.74,0.782778,0.985773
7,task-02,corruptions::gaussian_noise,task-02,94.54,0.184646,0.961213
8,task-02,corruptions::shot_noise,task-01,83.38,0.525784,0.995836
9,task-02,corruptions::shot_noise,task-02,96.03,0.123689,1.024094


In [ ]:
pd.read_csv('data/output/test/pmnist_robustness_test/perf_attack.csv')


,source_task,data_key,train_phase,epsilon,test_acc,test_time
0,task-01,adversarial::FGSM,task-01,0.0000,52.05,0.567941
1,task-01,adversarial::FGSM,task-01,0.0001,51.91,0.567941
2,task-01,adversarial::FGSM,task-01,0.0002,51.70,0.567941
3,task-01,adversarial::FGSM,task-01,0.0004,51.45,0.567941
4,task-01,adversarial::FGSM,task-01,0.0008,50.71,0.567941
...,...,...,...,...,...,...
415,task-02,adversarial::DDNAttack-2,task-02,0.6000,69.66,0.123821
416,task-02,adversarial::DDNAttack-2,task-02,0.7000,57.99,0.123821
417,task-02,adversarial::DDNAttack-2,task-02,0.8000,44.91,0.123821
418,task-02,adversarial::DDNAttack-2,task-02,0.9000,35.12,0.123821


In [ ]:
import plotly.express as px

px.line(
    df,
    x = 'epsilon',
    y = 'test_acc', 
    color = 'source_task',
    facet_col = 'data_key', 
    facet_row = 'train_phase',
    log_x=True
)